In [13]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [14]:
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [17]:
llm=ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.7)

In [18]:
llm

ChatGoogleGenerativeAI(model='gemini-pro', client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
))

In [21]:
with open("E:\MCQ_Generator\MCQ_Generator\Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [22]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [72]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs

{RESPONSE_JSON}

"""

In [73]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [74]:
quiz_generation_prompt

PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n\n{RESPONSE_JSON}\n\n')

In [75]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [28]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [76]:
TEMPLATE2

'\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. \nif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\nQuiz_MCQs:\n{quiz}\n\nCheck from an expert English Writer of the above quiz:\n'

In [77]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [78]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [79]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],output_variables=["quiz", "review"], verbose=True,)

In [80]:
PATH="E:\MCQ_Generator\MCQ_Generator\data.txt"

In [81]:
print(PATH)

E:\MCQ_Generator\MCQ_Generator\data.txt


In [82]:
with open(PATH,"r") as file:
    TEXT=file.read()

In [36]:
print(TEXT)

Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

In [83]:
generate_evaluate_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n\n{RESPONSE_JSON}\n\n'), llm=ChatGoogleGenerativeAI(model='gemini-pro', client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
)), output_key='quiz'), LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['quiz', 'subject'], template='\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexit

In [38]:
print(TEXT)

Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

In [84]:
TEXT
NUMBER=5
SUBJECT="AI"
TONE="Simple",
RESPONSE_JSON=RESPONSE_JSON

In [85]:
RESPONSE_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [93]:
import json
ab= json.dumps(RESPONSE_JSON)

In [94]:
 response=generate_evaluate_chain(
        {

        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": ab

        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large 

In [95]:
reviews = response.get("review")

In [96]:
reviews

"**Complexity Analysis:**\n\nThe quiz questions are clear, concise, and well-structured. They effectively assess the students' understanding of the workshop's topics. However, the quiz could be improved by including more challenging questions that require higher-order thinking skills.\n\n**Updated Quiz Questions:**\n\n1. Elaborate on the ethical considerations and potential societal implications of Generative AI.\n\n2. Analyze the strengths and limitations of different Generative AI techniques, such as LLMs, GANs, and VAEs.\n\n3. Discuss the role of data quality and diversity in training Generative AI models.\n\n4. Evaluate the potential impact of Generative AI on various industries and sectors.\n\n5. Propose innovative applications of Multi-modal Generative AI in fields such as healthcare, education, and entertainment.\n\n**Tone and Language:**\n\nThe tone of the quiz is professional, objective, and neutral. The language used is clear, concise, and appropriate for the target audience.

In [119]:
quiz=response.get("quiz")

In [125]:
quiz= '''  { "1": {\n        "no": "1",\n        "mcq": "What is the primary focus of the workshop on Generative AI at AIMLSystems?",\n        "options": {\n            "a": "Developing and deploying Generative AI systems",\n            "b": "Building and deploying models for realistic text generation",\n            "c": "Training and evaluating LLMs for image generation",\n            "d": "Evaluating the ethics and societal implications of Generative AI"\n        },\n        "correct": "a"\n    },\n    "2": {\n        "no": "2",\n        "mcq": "Which of the following is NOT a topic of interest for the workshop?",\n        "options": {\n            "a": "Systems, architecture and infrastructure for Generative AI",\n            "b": "Responsible use of Generative AI",\n            "c": "Quantum Computing and Generative AI",\n            "d": "Multi-modal Generative AI and its applications"\n        },\n        "correct": "c"\n    },\n    "3": {\n        "no": "3",\n        "mcq": "What is one of the key technologies used in Generative AI?",\n        "options": {\n            "a": "Large Language Models (LLMs)",\n            "b": "Convolutional Neural Networks (CNNs)",\n            "c": "Support Vector Machines (SVMs)",\n            "d": "Decision Trees"\n        },\n        "correct": "a"\n    },\n    "4": {\n        "no": "4",\n        "mcq": "What is a potential concern associated with Generative AI?",\n        "options": {\n            "a": "Fairness and bias",\n            "b": "Computational cost",\n            "c": "Data privacy",\n            "d": "All of the above"\n        },\n        "correct": "d"\n    },\n    "5": {\n        "no": "5",\n        "mcq": "What is one of the potential applications of Multi-modal Generative AI?",\n        "options": {\n            "a": "Creating personalized experiences",\n            "b": "Generating synthetic data",\n            "c": "Developing autonomous agents",\n            "d": "All of the above"\n        },\n        "correct": "d"\n    }\n} '''


In [126]:
quiz=json.loads(quiz)

In [128]:
quiz

{'1': {'no': '1',
  'mcq': 'What is the primary focus of the workshop on Generative AI at AIMLSystems?',
  'options': {'a': 'Developing and deploying Generative AI systems',
   'b': 'Building and deploying models for realistic text generation',
   'c': 'Training and evaluating LLMs for image generation',
   'd': 'Evaluating the ethics and societal implications of Generative AI'},
  'correct': 'a'},
 '2': {'no': '2',
  'mcq': 'Which of the following is NOT a topic of interest for the workshop?',
  'options': {'a': 'Systems, architecture and infrastructure for Generative AI',
   'b': 'Responsible use of Generative AI',
   'c': 'Quantum Computing and Generative AI',
   'd': 'Multi-modal Generative AI and its applications'},
  'correct': 'c'},
 '3': {'no': '3',
  'mcq': 'What is one of the key technologies used in Generative AI?',
  'options': {'a': 'Large Language Models (LLMs)',
   'b': 'Convolutional Neural Networks (CNNs)',
   'c': 'Support Vector Machines (SVMs)',
   'd': 'Decision Tr

In [62]:
reviews

'**Complexity Analysis:**\nThe quiz questions are straightforward and focus on the basic concepts of Generative AI, making them suitable for students with a fundamental understanding of the field.\n\n**Questions to be Updated:**\nNone of the questions need to be updated as they align well with the cognitive and analytical abilities of the students.\n\n**Tone and Language:**\nThe tone and language used in the quiz are clear, concise, and accessible, making them easy for students to understand and engage with.'

In [129]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [105]:
quiz_table_data

[{'MCQ': 'What is the primary focus of the workshop on Generative AI at AIMLSystems?',
  'Choices': 'a: Developing and deploying Generative AI systems | b: Building and deploying models for realistic text generation | c: Training and evaluating LLMs for image generation | d: Evaluating the ethics and societal implications of Generative AI',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a topic of interest for the workshop?',
  'Choices': 'a: Systems, architecture and infrastructure for Generative AI | b: Responsible use of Generative AI | c: Quantum Computing and Generative AI | d: Multi-modal Generative AI and its applications',
  'Correct': 'c'},
 {'MCQ': 'What is one of the key technologies used in Generative AI?',
  'Choices': 'a: Large Language Models (LLMs) | b: Convolutional Neural Networks (CNNs) | c: Support Vector Machines (SVMs) | d: Decision Trees',
  'Correct': 'a'},
 {'MCQ': 'What is a potential concern associated with Generative AI?',
  'Choices': 'a: Fairne

In [130]:
df=pd.DataFrame(quiz_table_data)

In [131]:
df.to_csv("genai.csv",index=False)

In [132]:
LOG_FILE="{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [133]:
LOG_FILE

"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [134]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'02_22_2024_03_14_05'